In [9]:
import pandas as pd

full = pd.read_csv('./results/Predictions_Full.csv')
lora = pd.read_csv('./results/Predictions_Lora.csv')


In [10]:
full.head()


,input_text,generated_text,reference_texts_0,reference_texts_1,reference_texts_2,predicted_label,reference_label
0,premise: This church choir sings to the masses...,The church choir can't sing joyous songs if th...,Not all churches have cracks in the ceiling,There is no indication that there are cracks i...,Not all churches have cracks in the ceiling.,contradiction,neutral
1,premise: This church choir sings to the masses...,A church choir sings joyous songs from the boo...,"""Filled with song"" is a rephrasing of the ""cho...",hearing song brings joyous in the church.,If the church choir sings then the church is f...,entailment,entailment
2,premise: This church choir sings to the masses...,A church choir is not a baseball game.,A choir sing some other songs other than book ...,The choir is at a chruch not a baseball game.,A baseball game isn’t played at a church.,contradiction,contradiction
3,"premise: A woman with a green headscarf, blue ...",Not all women are young.,the woman could've been old rather than young,There is no indication that the woman is young.,Not all women are young.,neutral,neutral
4,"premise: A woman with a green headscarf, blue ...",A woman with a big grin is not necessarily ver...,a grin suggests hapiness.,A WOMAN WITH BIG GRIN IS HAPPY,One must be happy in order to have a big grin.,entailment,entailment


# Label analysis

In [11]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

full_f1 = f1_score(full["reference_label"], full["predicted_label"], average="weighted")
lora_f1 = f1_score(lora["reference_label"], lora["predicted_label"], average="weighted")
full_acc = accuracy_score(full["reference_label"], full["predicted_label"])
lora_acc = accuracy_score(lora["reference_label"], lora["predicted_label"])
full_prec = precision_score(full["reference_label"], full["predicted_label"], average="weighted")
lora_prec = precision_score(lora["reference_label"], lora["predicted_label"], average="weighted")
full_rec = recall_score(full["reference_label"], full["predicted_label"], average="weighted")
lora_rec = recall_score(lora["reference_label"], lora["predicted_label"], average="weighted")

print(f"f1 lora={lora_f1} & f1 full={full_f1}")
print(f"acc lora={lora_acc} & acc full={full_acc}")
print(f"prec lora={lora_prec} & prec full={full_prec}")
print(f"recall lora={lora_rec} & recall full={full_rec}")


f1 lora=0.8832395589987994 & f1 full=0.9022181400980718
acc lora=0.8832451140065146 & acc full=0.902178338762215
prec lora=0.8832387590047901 & prec full=0.9023125710571344
recall lora=0.8832451140065146 & recall full=0.902178338762215


# Identify errors 

In [12]:
# create csvs with wrong answers
df_full = full.assign(correct=full.predicted_label == full.reference_label)
full_false = df_full[df_full['correct'] == False].drop('correct', axis=1)

df_lora = lora.assign(correct=lora.predicted_label == lora.reference_label)
lora_false = df_lora[df_lora['correct'] == False].drop('correct', axis=1)

full_false.to_csv('./results/False_Predictions_Full.csv', index=True)
lora_false.to_csv('./results/False_Predictions_Lora.csv', index=True)

# find items that one model gets correct whereas the other doesn't
lora_false_list = lora_false.index.tolist()
full_false_list = full_false.index.tolist()

not_in_lora = [x for x in full_false_list if x not in lora_false_list]
not_in_full = [x for x in lora_false_list if x not in full_false_list]

print(not_in_lora)
print(len(not_in_lora))

print(not_in_full)
print(len(not_in_full))



[5, 36, 37, 62, 78, 92, 148, 173, 237, 304, 457, 481, 546, 549, 645, 822, 833, 851, 953, 1041, 1101, 1126, 1137, 1163, 1169, 1203, 1206, 1222, 1270, 1325, 1346, 1356, 1417, 1525, 1533, 1540, 1557, 1585, 1591, 1802, 1871, 1895, 1924, 1927, 1978, 2014, 2030, 2050, 2119, 2132, 2208, 2234, 2284, 2318, 2425, 2563, 2620, 2621, 2671, 2700, 2730, 2793, 2860, 2939, 2948, 2974, 3019, 3061, 3062, 3070, 3203, 3226, 3233, 3273, 3324, 3350, 3491, 3565, 3826, 3847, 3969, 4046, 4087, 4123, 4133, 4136, 4273, 4303, 4327, 4356, 4444, 4483, 4555, 4574, 4611, 4631, 4706, 4726, 4821, 4860, 4879, 4888, 4924, 4965, 4989, 5094, 5110, 5164, 5273, 5300, 5366, 5430, 5449, 5459, 5472, 5490, 5620, 5890, 6024, 6132, 6183, 6226, 6276, 6345, 6372, 6445, 6466, 6467, 6501, 6570, 6683, 6741, 6779, 6820, 6891, 6931, 6934, 6962, 6988, 7013, 7021, 7022, 7034, 7047, 7067, 7093, 7137, 7229, 7264, 7335, 7365, 7378, 7385, 7424, 7461, 7501, 7504, 7513, 7522, 7528, 7535, 7592, 7628, 7635, 7642, 7675, 7705, 7716, 7744, 7788, 7815,

# Analyse occurance of negation

In [55]:
pattern = r'(.+?) (not) (.+)'

all_count = full['generated_text'].str.count(pattern).sum()
false_count = full_false['generated_text'].str.count(pattern).sum()

total = len(full['generated_text'])
print(total)
print(f'The pattern "{pattern}" occurs {all_count} times in the "generated_text" column: {all_count/total}%..')

total_false = len(full_false['generated_text'])
print(total_false)
print(f'The pattern "{pattern}" occurs {false_count} times in the "generated_text" column: {false_count/total_false}%.')


9824
The pattern "(.+?) (not) (.+)" occurs 3665 times in the "generated_text" column: 0.37306596091205213%..
961
The pattern "(.+?) (not) (.+)" occurs 461 times in the "generated_text" column: 0.4797086368366285%.
